definition of conctact model 

In [245]:
class ContactDetail:
    def __init__(self, tag: str, detail: str):
        self.tag = tag
        self.detail = detail

class Contact:
    def __init__(self, id: int, family_name: str = "", first_name: str = ""):
        self.id = id
        self.family_name = family_name
        self.first_name = first_name
        self.details = []

data structures needed for contact management

In [259]:
from typing import Iterable

class binary_tree_node:
    def __init__(self, contact: Contact):
        self.contact = contact
        self.left = None
        self.right = None

    def assign(self, node):
        self.contact = node.contact
        self.right = node.right
        self.left = node.left

class naive_hasher:
    def hash(self, id: int) -> int:
        return id

class binary_tree:
    def __init__(self, hasher = naive_hasher()):
        self.__head = None
        self.hasher = hasher

    def getById(self, id: int) -> Contact:
        hash = self.hasher.hash(id)

        node = self.__head
        while node is not None:
            probe = self.hasher.hash(node.contact.id)
            if probe == hash:
                return node.contact
            elif hash < probe:
                node = node.left
            else:
                node = node.right
        return None

    def insert(self, contact: Contact):
        node = binary_tree_node(contact)
        if self.__head is None:
            self.__head = node
            return
        
        hash = self.hasher.hash(node.contact.id)
        return self.__insertAt(node, hash, self.__head)
    
    def remove(self, contact: Contact):
        hash = self.hasher.hash(contact.id)

        parent = None
        node = self.__head
        while node is not None:
            probe = self.hasher.hash(node.contact.id)
            if probe == hash:
                if node.right is not None:
                    left = node.left
                    node.assign(node.right) # reference to node.right now gone. GC will cleanup

                    # relocating the left node
                    if left is not None:
                        self.__insertAt(left, self.hasher.hash(left.contact.id), node)
                elif node.left is not None:
                    node.assign(node.left) # similiar, promote the child and forget original reference 
                else:
                    if parent == None:
                        self.__head = None
                    elif parent.left == node:
                        parent.left = None
                    else:
                        parent.right = None
                return True

            parent = node
            if hash < probe:
                node = node.left
            else:
                node = node.right
        return False

    # traverses the tree with left-first strategy: left-leaf > node > right-leaf
    def travers(self, node: binary_tree_node = None) -> Iterable[Contact]:
        if node is None:
            if self.__head is None:
                return
            node = self.__head

        if node.left is not None:
            yield from self.travers(node.left)
        yield node.contact
        if node.right is not None:
            yield from self.travers(node.right)

    def __insertAt(self, node: binary_tree_node, nodeHash, at: binary_tree_node):
        probe = self.hasher.hash(at.contact.id)
        if probe == nodeHash:
            # Contact with same ID already exists
            return False
        
        if nodeHash < probe:
            if at.left is None:
                at.left = node
                return True
            return self.__insertAt(node, nodeHash, at.left)
        else:
            if at.right is None:
                at.right = node
                return True
            return self.__insertAt(node, nodeHash, at.right)

In [257]:
# test for insertion
tree = binary_tree()
tree.insert(Contact(20))  #           20
tree.insert(Contact(10))  #      10        25
tree.insert(Contact(5))   #   5     15         35
tree.insert(Contact(15)) 
tree.insert(Contact(25))
tree.insert(Contact(35))
assert [x.id for x in tree.travers()] == [5, 10, 15, 20, 25, 35]

tree.insert(Contact(26))
assert [x.id for x in tree.travers()] == [5, 10, 15, 20, 25, 26, 35]

assert tree.insert(Contact(26)) == False # already exists

In [258]:
# test for deletion (same tree as insertion test) 
tree = binary_tree()
tree.insert(Contact(20))  #           20
tree.insert(Contact(10))  #      10        25
tree.insert(Contact(5))   #   5     15         35
tree.insert(Contact(15)) 
tree.insert(Contact(25))
tree.insert(Contact(35))

tree.remove(Contact(20))
#           25
#      10        35
#   5     15         
assert [x.id for x in tree.travers()] == [5, 10, 15, 25, 35]

tree.remove(Contact(10))
tree.remove(Contact(5))
#           25
#      15        35
assert [x.id for x in tree.travers()] == [15, 25, 35]

# test for deletion 2 (left-node relocation)
tree = binary_tree()
tree.insert(Contact(20))  #    20
tree.insert(Contact(25))  #       25
tree.insert(Contact(37))  #           37
tree.insert(Contact(40))  #       34      40
tree.insert(Contact(34))  #     33  35  38 
tree.insert(Contact(33))  
tree.insert(Contact(38))  
tree.insert(Contact(35))
assert [x.id for x in tree.travers()] == [20, 25, 33, 34, 35, 37, 38, 40]

tree.remove(Contact(37))
#    20
#       25
#          40
#       38
#    34
#  33  35
assert [x.id for x in tree.travers()] == [20, 25, 33, 34, 35, 38, 40]

# test for deletion (cleanup head)
tree = binary_tree()
tree.insert(Contact(35))
tree.remove(Contact(35))
assert [x.id for x in tree.travers()] == []

In [260]:
class dictionary:
    def __init__(self, hasher = naive_hasher()):
        self.numBuckets = 64
        self.buckets = [binary_tree() for _ in range(self.numBuckets)]
        self.hasher = hasher

    def insert(self, c: Contact):
        index = self.hasher.hash(c.id) % self.numBuckets
        bucket = self.buckets[index]
        bucket.insert(c)

    def remove(self, c: Contact):
        index = self.hasher.hash(c.id) % self.numBuckets
        bucket = self.buckets[index]
        bucket.remove(c)

    def get(self, id: int) -> Contact:
        index = self.hasher.hash(id) % self.numBuckets
        bucket = self.buckets[index]
        return bucket.getById(id)

In [262]:
# test dictionary insert/remove/get

d = dictionary()
d.insert(Contact(15, "foo", "bar"))
c = d.get(15)

assert c.family_name == "foo" and c.first_name == "bar"

TypeError: unsupported operand type(s) for %: 'Contact' and 'int'

definition of model: contact, contact details, and contact book

In [ ]:
def stringContains(text: str, subText: str) -> bool:
    for i in range(len(text)):
        found = True 
        for j in range(len(subText)):
            if subText[j] != text[i+j]:
                found = False
                break
        if found:
            return True
    return False

    def Search(contacts: Iterable[Contact], name: str) -> Iterable[Contact]:
        result = []
        for c in contacts:
            match = stringContains(c.firstName, name) or stringContains(c.lastName, name)
            if match:
                result.append(c)
        return result

class ContactBook:
    def __init__(self):
        self.__id = 1000
        self.__contacts = {} 

    def add(self, fam_name: str, first_name: str) -> int:
        id = self.__id
        self.__id = self.__id + 1
        self.__contacts[id] = Contact(id, fam_name, first_name)
        return id

    def contacts(self) -> Iterable[Contact]:
        return self.__contacts.values()